VQE：变分法

VQE本身就有很大的自由度，可以选择不同的线路，这里使用了组里常用的线路，请按需修改

这里用了类（class），请自行bing搜索相关使用方法

这里是最小化自由能，如果需要最小化其它，请自行修改

In [ ]:
from quspin.operators import hamiltonian  # Hamiltonians and operators
from quspin.basis import spin_basis_1d  # Hilbert space spin basis
from quspin.tools.evolution import expm_multiply_parallel
import numpy as np  # generic math functions
import matplotlib.pyplot as plt
from scipy.optimize import minimize, basinhopping
from datetime import date
import multiprocessing as mp
from scipy import sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigs
import random
from math import pi

In [ ]:
# define one Pauli matrices
def h_n(Interaction_form: str, position: int, Coefficient=1):
    """this function is one Pauli matrices, return hamiltonian.
    need choose basis first!!!
    first parameter is interaction form, you can input 'x','y','z','I'.
    second parameter is coefficient before,
    last two parameter is lattice position,
     """
    h_list = [[Coefficient, position]]
    h_op = [[Interaction_form, h_list]]
    dynamic = []
    ham = hamiltonian(static_list=h_op, dynamic_list=dynamic,
                      dtype=np.complex128, basis=basis, check_herm=False, check_symm=False)
    return ham

# define two Pauli matrices interaction
def h_nn(Interaction_form: str, position1: int, position2: int, Coefficient=1):
    """this function is two Pauli matrices interaction, return hamiltonian.
    need choose basis first!!!
    first parameter is interaction form, you can input 'xx','yy','zz'...
    second parameter is coefficient before,
    last two parameter is lattice position,
     """
    h_list = [[Coefficient, position1, position2]]
    h_op = [[Interaction_form, h_list]]
    dynamic = []
    ham = hamiltonian(static_list=h_op, dynamic_list=dynamic,
                      dtype=np.complex128, basis=basis, check_herm=False, check_symm=False)
    return ham

In [ ]:
class VQE(): 
    def __init__(self,state,layers,temperature):
        """需要输入初态，运行层数和温度"""
        self.layers = layers
        self.state = state
        self.temperature = temperature

    def CNOT(self,position_i:int,position_j:int): # 返回数据类型：expm_multiply_parallel
        """position_i is control qbit!"""
        temp = h_nn('II',position1=position_i,position2=position_j) - h_nn('zI',position1=position_i,position2=position_j) - h_nn('Ix',position1=position_i,position2=position_j) + h_nn('zx',position1=position_i,position2=position_j)
        res = expm_multiply_parallel(A=temp.tocsc(),a=1j*pi/4)
        return res
    
    def circuit1(self,alpha): # 制造混态（m+1个为一组）
        """alpha大小为1*n（=辅助qubit数量）"""
        for i in range(n): # 旋转辅助比特(RX)，i为第i个辅助qubit
            self.state = expm_multiply_parallel(A=h_n('x',n*m+i).tocsr(),a=-1j * alpha[i]).dot(self.state)
        for i in range(n): # 作用CNOT
            for j in range(m):
                self.state = self.CNOT(n*m+i,m*i+j).dot(self.state)

    def circuit2(self,beta,psi_index): # 对混态进行演化，beta为一个长度为5*l的参数表（演化参数）
        """beta大小为3*l，3 from Hamilton, and l are layers"""
        if psi_index==1:
            for l in range(self.layers): # 演化层数
                self.state = expm_multiply_parallel(A = Hx1.tocsr(), a = -1j * beta[H_number*l]).dot(self.state)
                self.state = expm_multiply_parallel(A = Hy1.tocsr(), a = -1j * beta[H_number*l+1]).dot(self.state)
                # self.state = expm_multiply_parallel(A = Hm1.tocsr(), a = -1j * beta[H_number*l+2]).dot(self.state)
                self.state = expm_multiply_parallel(A = HI1.tocsr(), a = -1j * beta[H_number*l+2]).dot(self.state)
        elif psi_index==2:
            for l in range(self.layers): # 演化层数
                self.state = expm_multiply_parallel(A = Hx2.tocsr(), a = -1j * beta[H_number*l]).dot(self.state)
                self.state = expm_multiply_parallel(A = Hy2.tocsr(), a = -1j * beta[H_number*l+1]).dot(self.state)
                # self.state = expm_multiply_parallel(A = Hm2.tocsr(), a = -1j * beta[H_number*l+2]).dot(self.state)
                self.state = expm_multiply_parallel(A = HI2.tocsr(), a = -1j * beta[H_number*l+2]).dot(self.state)

    def thermal_state(self,params,psi_index):
        """返回做完线路之后的态"""
        alpha = params[0 : n]
        beta = params[n : n + H_number*self.layers]
        self.circuit1(alpha=alpha)
        self.circuit2(beta=beta,psi_index=psi_index)
        return self.state
    
    def entropy(self,params):
        res = 0
        for i in range(n):
            temp_a = params[i]
            res += - np.sin(temp_a)**2 * np.log(np.sin(temp_a)**2) - np.cos(temp_a)**2 * np.log(np.cos(temp_a)**2)
        return res
    
    def energy(self,params,psi_index):
        if psi_index==1:
            temp_state = self.thermal_state(params=params,psi_index=1)
            res = np.vdot(temp_state,H1.dot(temp_state))
        elif psi_index==2:
            temp_state = self.thermal_state(params=params,psi_index=2)
            res = np.vdot(temp_state,H2.dot(temp_state))
        else:
            print('psi index out of range')
        return np.real(res)
    
    def free_energy(self,params,psi_index):
        E = self.energy(params=params,psi_index=psi_index)
        T = self.temperature
        S = self.entropy(params=params)
        return np.real(E - T * S)
    
    def Opt(self,psi_index):
        """only fix temperture!!!!"""
        params = np.random.rand(n+H_number*self.layers)
        optimizer = minimize(fun=self.free_energy, x0=params, args=psi_index, method='BFGS', options={'gtol': 1e-15, 'eps': 1e-08, 'disp': True}) # 'gtol'：梯度，'eps'：步长，'disp'：打印信息
        return optimizer
